In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
import os
os.chdir('/content/drive/My Drive/CausalDL/')

In [63]:
!nvidia-smi

Fri May  6 17:09:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    41W / 300W |    627MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load LIDH Data

In [64]:
from utils.data_reader import read_IHDP_data, read_SIPP_data

#data=read_SIPP_data(file = './data/SIPP/sipp1991.dta')
data=read_IHDP_data(training_data='./data/IHDP/ihdp_npci_1-100.train.npz',testing_data='./data/IHDP/ihdp_npci_1-100.test.npz')

In [65]:
!pip install -q tensorflow==2.8.0
import tensorflow as tf
import numpy as np
import datetime
#Colab command to allow us to run Colab in TF2
%load_ext tensorboard 

# Install Keras Tuner
!pip install keras-tuner==1.0.4

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [98]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN

from model.model_loss import Base_Dragon_Loss, TarReg_Loss, MSE_Loss, CFRNet_Loss
from model.model_metrics import AIPW_Metrics, TarReg_Metrics, Base_Metrics

In [99]:
def compute_prediction(data, model):
    """
    Compute model predicted ATE(average treatment effect) and compare to ground truth (if exists)
    """
    concat_pred=model.predict(data['x'])
    #dont forget to rescale the outcome before estimation!
    y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
    
    #predicted CATE
    cate_pred=y1_pred-y0_pred
    
    data_save = {}      
    data_save['cate_pred'] = cate_pred
    
    if 'mu_1' in data.keys() or 'mu_0' in data.keys():
        cate_true=data['mu_1']-data['mu_0'] #Hill's noiseless true values
        data_save['cate_true'] = cate_true
        print("Actual ATE:", cate_true.mean(),'\n\n')
    ate_pred=tf.reduce_mean(cate_pred)
    print("Estimated ATE:", ate_pred.numpy(),'\n\n')
    data_save['ate_pred'] = ate_pred
    
    return data_save

## Fine tune CFRNet

In [100]:
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch

def make_hypercfrnet(hp):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """
    # hp.Choice takes hyperparam name, list of options, and default
    reg_l2=hp.Choice('l2',[.1,.01,.001],default=.01)
    input_dim=25
    inputs = Input(shape=(input_dim,), name='input')

    # representation
    rep_units = hp.Choice('rep_units', [20, 25, 50,100],default=25)
    phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(inputs)
    for i in range(hp.Int('rep_layers', 1, 2, default=1)):
      #pretty nifty way to dynamically add more layers!
      phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_'+str(i+2))(phi)

    # HYPOTHESIS
    hyp_units = hp.Choice('hyp_units', [20,25, 50,100],default=25)
    y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)
    for i in range(hp.Int('hyp_layers', 1, 3, default=2)):
      y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_'+str(i+2))(y0_hidden)
      y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_'+str(i+2))(y1_hidden)
    
    # OUTPUT
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,phi])
    model = Model(inputs=inputs, outputs=concat_pred)
    
    sgd_lr = 1e-5
    #momentum = 0.9
    
    #optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True)
    cfrnet_loss=CFRNet_Loss(alpha=1.0)
    model.compile(optimizer=Adam(learning_rate=sgd_lr),
                       loss=cfrnet_loss,
                  metrics=[cfrnet_loss,cfrnet_loss.regression_loss,cfrnet_loss.mmdsq_loss])
    # model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
    #                   loss=regression_loss,
    #              metrics=regression_loss)


    return model

In [101]:
from keras_tuner.engine import tuner_utils
from tensorboard.plugins.hparams import api as hparams_api
!rm -rf cfr_dir

class CFRNetTuner(kt.Tuner):

  def run_trial(self, trial,dataset,*fit_args, **fit_kwargs):
      # *args and **kwargs in Python are positional (list) and keyword (dict) arguments
      verbose = fit_kwargs['verbose']

      log_dir=self.project_dir+'/trial_'+trial.trial_id
      hp = trial.hyperparameters
      
      batch_size = hp.Int('batch_size', 64, 256, step=64, default=64)
      stopping_patience=hp.Int('batch_size', 5, 15, step=5, default=5)

      #some of this hacky code will hopefully go away as Keras Tuner get's more polished
      hparams = tuner_utils.convert_hyperparams_to_hparams(trial.hyperparameters)
      file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
      file_writer.set_as_default()
      
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      hparams_callback = hparams_api.KerasCallback(
                        writer=log_dir,
                        hparams=hparams,
                        #I prepend trial_ here to get it to save nicely. Hopefully will be fixed in future version of KT.
                        trial_id='trial_'+trial.trial_id) 
      metrics_callback=Base_Metrics(dataset,verbose=verbose)
      callbacks = [
              TerminateOnNaN(),
              EarlyStopping(monitor='val_loss', patience=stopping_patience, min_delta=0.0001),
              ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                                min_delta=0, cooldown=0, min_lr=0),
              metrics_callback,
              tensorboard_callback,
              hparams_callback
          ]

      
      model = self.hypermodel.build(hp)
      model.fit(x=fit_args[0],y=fit_args[1],
                 callbacks=callbacks,
                  validation_split=fit_kwargs['validation_split'],
                  epochs=fit_kwargs['epochs'],
                  batch_size=batch_size, verbose=verbose)
      
      #give the metric to the hyperparameter optimization algorithm
      concat_pred=model.predict(data['x'])
      pehe_nn=metrics_callback.PEHEnn(concat_pred)
      self.oracle.update_trial(trial.trial_id, {'cate_nn_err': tf.sqrt(pehe_nn)})
      self.save_model(trial.trial_id, model)


In [ ]:
yt = np.concatenate([data['ys'], data['t']], 1)

tuner = CFRNetTuner(
    #the oracle is the hyperoptimization algorithm
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('cate_nn_err', 'min'),
        max_trials=10, #were trying to keep this quick for you.
        #You probably want to do as many trials as your resources allow if you see variance between runs
        seed=0    
),
        directory='cfr_dir',
        project_name='cfr_tune',
    hypermodel=make_hypercfrnet
    )
tuner.search(data, data['x'],yt, epochs=500,validation_split=.2,verbose=2)

best_trial=tuner.oracle.get_best_trials(num_trials=1)[0]
print("BEST TRIAL ID:",best_trial.trial_id)
best_model=tuner.load_model(best_trial)

In [109]:
best_trial.hyperparameters.values

{'batch_size': 64,
 'hyp_layers': 2,
 'hyp_units': 25,
 'l2': 0.1,
 'rep_layers': 1,
 'rep_units': 50}

In [ ]:
#save predicted result
data_save_cfr=compute_prediction(data, best_model)
np.save('./save/pred_result/tune_result_{}.npy'.format("cfrnet" + '_' + "IHDP"), data_save_cfr) 

## Fine Tune DragonNet

In [94]:
#https://github.com/kochbj/Deep-Learning-for-Causal-Inference

def make_hyperdragonnet(hp):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """
    # hp.Choice takes hyperparam name, list of options, and default
    reg_l2=hp.Choice('l2',[.1,.01,.001],default=.01)
    input_dim=25
    inputs = Input(shape=(input_dim,), name='input')

    # representation
    rep_units = hp.Choice('rep_units', [50,100,200],default=200)
    phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(inputs)
    for i in range(hp.Int('rep_layers', 1, 2, 3, default=1)):
      #pretty nifty way to dynamically add more layers!
      phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_'+str(i+2))(phi)

    # HYPOTHESIS
    hyp_units = hp.Choice('hyp_units', [20,50,100,200],default=100)
    y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)
    for i in range(hp.Int('hyp_layers', 1, 3, default=2)):
      y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_'+str(i+2))(y0_hidden)
      y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_'+str(i+2))(y1_hidden)
    
    # OUTPUT
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)
    
    t_prediction = Dense(units=1,activation=None,name='t_prediction')(phi)
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,t_prediction, phi])
    model = Model(inputs=inputs, outputs=concat_pred)
    
    sgd_lr = 1e-5
    momentum = 0.9
    
    optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True)
    aipw_loss=Base_Dragon_Loss(alpha=1.0)
    model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
                      loss=aipw_loss,
                 metrics=[aipw_loss,aipw_loss.regression_loss,aipw_loss.treatment_acc])

    return model

In [95]:
from keras_tuner.engine import tuner_utils
from tensorboard.plugins.hparams import api as hparams_api
!rm -rf dragon_dir

class DragonNetTuner(kt.Tuner):

  def run_trial(self, trial,dataset,*fit_args, **fit_kwargs):
      # *args and **kwargs in Python are positional (list) and keyword (dict) arguments
      verbose = fit_kwargs['verbose']

      log_dir=self.project_dir+'/trial_'+trial.trial_id
      hp = trial.hyperparameters
      
      batch_size = hp.Int('batch_size', 64, 256, step=64, default=64)
      stopping_patience=hp.Int('batch_size', 5, 15, step=5, default=5)

      #some of this hacky code will hopefully go away as Keras Tuner get's more polished
      hparams = tuner_utils.convert_hyperparams_to_hparams(trial.hyperparameters)
      file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
      file_writer.set_as_default()
      
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      hparams_callback = hparams_api.KerasCallback(
                        writer=log_dir,
                        hparams=hparams,
                        #I prepend trial_ here to get it to save nicely. Hopefully will be fixed in future version of KT.
                        trial_id='trial_'+trial.trial_id) 
      metrics_callback=AIPW_Metrics(dataset,verbose=verbose)
      callbacks = [
              TerminateOnNaN(),
              EarlyStopping(monitor='val_loss', patience=stopping_patience, min_delta=0.),
              ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                                min_delta=0, cooldown=0, min_lr=0),
              metrics_callback,
              tensorboard_callback,
              hparams_callback
          ]

      
      model = self.hypermodel.build(hp)
      model.fit(x=fit_args[0],y=fit_args[1],
                 callbacks=callbacks,
                  validation_split=fit_kwargs['validation_split'],
                  epochs=fit_kwargs['epochs'],
                  batch_size=batch_size, verbose=verbose)
      
      #give the metric to the hyperparameter optimization algorithm
      concat_pred=model.predict(data['x'])
      pehe_nn=metrics_callback.PEHEnn(concat_pred)
      self.oracle.update_trial(trial.trial_id, {'cate_nn_err': tf.sqrt(pehe_nn)})
      self.save_model(trial.trial_id, model)


In [96]:
tuner = DragonNetTuner(
    #the oracle is the hyperoptimization algorithm
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('cate_nn_err', 'min'),
        max_trials=10, #were trying to keep this quick for you.
        #You probably want to do as many trials as your resources allow if you see variance between runs
        seed=0    
),
        directory='dragon_dir',
        project_name='helloworld',
    hypermodel=make_hyperdragonnet
    )
tuner.search(data, data['x'],yt, epochs=300,validation_split=.2,verbose=2)

best_trial=tuner.oracle.get_best_trials(num_trials=1)[0]
print("BEST TRIAL ID:",best_trial.trial_id)
best_dragon_model=tuner.load_model(best_trial)

Trial 10 Complete [00h 01m 14s]
cate_nn_err: 1.730276346206665

Best cate_nn_err So Far: 1.5806220769882202
Total elapsed time: 00h 15m 20s
INFO:tensorflow:Oracle triggered exit
BEST TRIAL ID: 56df958159155b3dcb38156758f63143


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [116]:
best_dragon_model.get_config()

{'input_layers': [['input', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 25),
    'dtype': 'float32',
    'name': 'input',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input'},
  {'class_name': 'Dense',
   'config': {'activation': 'elu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'RandomNormal',
     'config': {'mean': 0.0, 'seed': None, 'stddev': 0.05}},
    'kernel_regularizer': None,
    'name': 'phi_1',
    'trainable': True,
    'units': 50,
    'use_bias': True},
   'inbound_nodes': [[['input', 0, 0, {}]]],
   'name': 'phi_1'},
  {'class_name': 'Dense',
   'config': {'activation': 'elu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros',

In [97]:
#save predicted result
data_save_dragon=compute_prediction(data, best_dragon_model)
np.save('./save/pred_result/tune_result_{}.npy'.format("dragonnet" + '_' + "IHDP"), data_save_dragon) 

Actual ATE: 3.8536534 


Estimated ATE: 3.7764614 




## Fine-tune TARNet

In [84]:
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
def make_hypertarnet(hp):
    """
    Neural net predictive model. The dragon has three heads.
    :param input_dim:
    :param reg:
    :return:
    """
    # hp.Choice takes hyperparam name, list of options, and default
    reg_l2=hp.Choice('l2',[.1,.01,.001],default=.01)
    input_dim=25
    inputs = Input(shape=(input_dim,), name='input')

    # representation
    rep_units = hp.Choice('rep_units', [50,100,200],default=200)
    phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(inputs)
    for i in range(hp.Int('rep_layers', 1, 2,3, default=1)):
      #pretty nifty way to dynamically add more layers!
      phi = Dense(units=rep_units, activation='elu', kernel_initializer='RandomNormal',name='phi_'+str(i+2))(phi)

    # HYPOTHESIS
    hyp_units = hp.Choice('hyp_units', [20,50,100,200],default=100)
    y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)
    for i in range(hp.Int('hyp_layers', 1, 3, default=2)):
      y0_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_'+str(i+2))(y0_hidden)
      y1_hidden = Dense(units=hyp_units, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_'+str(i+2))(y1_hidden)
    
    # OUTPUT
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,phi])
    model = Model(inputs=inputs, outputs=concat_pred)
    
    sgd_lr = 1e-5
    momentum = 0.9
    
    optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True)
    
    model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
                      loss=regression_loss,
                 metrics=regression_loss)

    return model

In [91]:
from keras_tuner.engine import tuner_utils
from tensorboard.plugins.hparams import api as hparams_api
!rm -rf tar_dir
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]
 
    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]
 
    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses even though mathematically we should be using the mean
    #tf.reduce_mean and tf.reduce_sum should be equivalent, but maybe having larger error gradients makes training easier?
    return loss0 + loss1

class TarNetTuner(kt.Tuner):

  def run_trial(self, trial,dataset,*fit_args, **fit_kwargs):
      # *args and **kwargs in Python are positional (list) and keyword (dict) arguments
      verbose = fit_kwargs['verbose']

      log_dir=self.project_dir+'/trial_'+trial.trial_id
      hp = trial.hyperparameters
      
      batch_size = hp.Int('batch_size', 64, 256, step=64, default=64)
      stopping_patience=hp.Int('batch_size', 5, 15, step=5, default=5)

      #some of this hacky code will hopefully go away as Keras Tuner get's more polished
      hparams = tuner_utils.convert_hyperparams_to_hparams(trial.hyperparameters)
      file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
      file_writer.set_as_default()
      
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      hparams_callback = hparams_api.KerasCallback(
                        writer=log_dir,
                        hparams=hparams,
                        #I prepend trial_ here to get it to save nicely. Hopefully will be fixed in future version of KT.
                        trial_id='trial_'+trial.trial_id) 
      metrics_callback=Base_Metrics(dataset,verbose=verbose)
      callbacks = [
              TerminateOnNaN(),
              EarlyStopping(monitor='val_loss', patience=stopping_patience, min_delta=0.0001),
              ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                                min_delta=0, cooldown=0, min_lr=0),
              metrics_callback,
              tensorboard_callback,
              hparams_callback
          ]

      
      model = self.hypermodel.build(hp)
      model.fit(x=fit_args[0],y=fit_args[1],
                 callbacks=callbacks,
                  validation_split=fit_kwargs['validation_split'],
                  epochs=fit_kwargs['epochs'],
                  batch_size=batch_size, verbose=verbose)
      
      #give the metric to the hyperparameter optimization algorithm
      concat_pred=model.predict(data['x'])
      pehe_nn=metrics_callback.PEHEnn(concat_pred)
      self.oracle.update_trial(trial.trial_id, {'cate_nn_err': tf.sqrt(pehe_nn)})
      self.save_model(trial.trial_id, model)


In [92]:
tuner = TarNetTuner(
    #the oracle is the hyperoptimization algorithm
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective('cate_nn_err', 'min'),
        max_trials=10, #were trying to keep this quick for you.
        #You probably want to do as many trials as your resources allow if you see variance between runs
        seed=0    
),
        directory='tar_dir',
        project_name='tar_tune',
    hypermodel=make_hypertarnet
    )
tuner.search(data, data['x'],yt, epochs=300,validation_split=.2,verbose=2)

best_trial=tuner.oracle.get_best_trials(num_trials=1)[0]
print("BEST TRIAL ID:",best_trial.trial_id)
best_tar_model=tuner.load_model(best_trial)

Trial 10 Complete [00h 01m 23s]
cate_nn_err: 1.5489557981491089

Best cate_nn_err So Far: 1.5489557981491089
Total elapsed time: 00h 13m 46s
INFO:tensorflow:Oracle triggered exit
BEST TRIAL ID: 130b7323bdcb7146bfbd44961b2390fb


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [117]:
best_tar_model.get_config()

{'input_layers': [['input', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 25),
    'dtype': 'float32',
    'name': 'input',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input'},
  {'class_name': 'Dense',
   'config': {'activation': 'elu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'RandomNormal',
     'config': {'mean': 0.0, 'seed': None, 'stddev': 0.05}},
    'kernel_regularizer': None,
    'name': 'phi_1',
    'trainable': True,
    'units': 50,
    'use_bias': True},
   'inbound_nodes': [[['input', 0, 0, {}]]],
   'name': 'phi_1'},
  {'class_name': 'Dense',
   'config': {'activation': 'elu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros',

In [93]:
#save predicted result
data_save_tar=compute_prediction(data, best_tar_model)
np.save('./save/pred_result/tune_result_{}.npy'.format("tarnet" + '_' + "IHDP"), data_save_tar) 

Actual ATE: 3.8536534 


Estimated ATE: 3.7309685 


